# Query the Senator Knowledge Graph

This notebook provides the code for various querying of the senator knowledge graph built in build_graph.ipynb

In [1]:
import pandas as pd
import neo4j
import seaborn as sns

In [2]:
driver = neo4j.GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "senate"))

In [3]:

def run(query):
    data = []
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            data.append(record.data())
    return pd.DataFrame(data)

# Example Queries

## --- Simple Queries ---

### (1) Find senators who aren't republicans or democrats

In [4]:
run("""
MATCH (s: Senator {party: "other"})
RETURN s.name, s.party, s.id
""")

,s.name,s.party,s.id
0,Angus King,other,41300
1,Bernie Sanders,other,29147


### (2) Get roll calls with over 90 yea votes

In [5]:
run("""
MATCH (r: Rollcall)
WHERE r.yea_count > 90
RETURN r.rollnumber, r.desc, r.yea_count
ORDER by r.yea_count DESC
""")

,r.rollnumber,r.desc,r.yea_count
0,16,To establish a deficit-neutral reserve fund re...,100
1,21,To establish a deficit-neutral reserve fund re...,100
2,27,To establish a deficit-neutral reserve fund re...,100
3,35,To establish a deficit-neutral fund relating t...,100
4,272,A bill to deposit certain funds into the Crime...,100
5,659,A bill to suspend normal trade relations treat...,100
6,660,A bill to prohibit the importation of energy p...,100
7,19,To establish a deficit-neutral reserve fund re...,99
8,26,To establish a deficit-neutral reserve fund re...,99
9,316,To prohibit enactment of the Green New Deal.,99


### (3) Get all bills Bernie Sanders voted yea on

In [6]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[v:Voted {vote_type: "yea"}]-(r:Rollcall)
RETURN v.vote_type, r.desc, r.bill_number
""")

,v.vote_type,r.desc,r.bill_number
0,yea,A bill to authorize funds for Federal-aid high...,HR3684
1,yea,"Rashad Hussain, of Virginia, to be Ambassador ...",PN1006
2,yea,"Sean C. Staples, of the District of Columbia, ...",PN800
3,yea,"Chris Magnus, of Arizona, to be Commissioner o...",PN565
4,yea,"Jennifer Sung, of Oregon, to be United States ...",PN807
...,...,...,...
548,yea,A joint resolution providing for congressional...,SJRES14
549,yea,"Radhika Fox, of California, to be an Assistant...",PN444
550,yea,"Sherilyn Peace Garnett, of California, to be U...",PN1478
551,yea,"Zahid N. Quraishi, of New Jersey, to be United...",PN394


### (4) Get all of Bernie Sander tweets ordered by likes

In [7]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[tw:Tweeted]->(t:Tweet)
RETURN t.text, t.likes
ORDER BY t.likes DESC
""")

,t.text,t.likes
0,Let's be clear. Eli Lilly should apologize for...,481505
1,"The people of Brazil have voted for democracy,...",180877
2,Progressive candidates had a great election ni...,39733
3,What's going on in this country right now is c...,25880
4,I do not think the American people should be p...,22300
...,...,...
95,RT @NextGenAmerica: “Too many Americans have f...,0
96,RT @SBWorkersUnited: RED CUP REBELLION. OVER 1...,0
97,RT @BernieSanders: OUR FUTURE IS NOW: Our job ...,0
98,RT @BernieSanders: LIVE FROM TEXAS: @GregCasar...,0


### (5) Get the named entities that were contributors to senators

In [8]:
run("""
MATCH (e:Entity)-[sa:SameAs]->(c:Contributor)
RETURN e.name
""")

,e.name
0,u.s. army
1,university of virginia
2,bae systems
3,bath iron works
4,lockheed martin
5,wells fargo
6,planned parenthood
7,express scripts
8,brown university
9,citizens united


## --- Intermediate Level Queries ---

### (6) Get the organization entities mentioned in roll calls for which Bernie Sander voted yea on, and order by organization count 

In [9]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[v:Voted {vote_type: "yea"}]->(r:Rollcall)-[m:Mentions]->(e:Entity {type: "ORG"})
RETURN e.name, count(e)
ORDER BY count(e) DESC
""")

,e.name,count(e)
0,state,18
1,treasury,13
2,defense,12
3,the environmental protection agency,12
4,washington,11
...,...,...
133,medicaid,1
134,the office of minority economic impact,1
135,the internal revenue service,1
136,the national commission to investigate the jan...,1


### (7) Get the organization entities that Bernie Sanders tweets about, and order by organization count

In [10]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[tw:Tweeted]->(t:Tweet)-[m:Mentions]->(e:Entity {type: "ORG"})
RETURN e.name, count(e)
ORDER BY count(e) DESC
""")

,e.name,count(e)
0,congress,13
1,texas,11
2,washington,3
3,senate,3
4,nevada,3
5,starbucks,2
6,the u.s. house,2
7,@summerforpa,2
8,medicare for all,2
9,social security,2


### (8) Get the event entities that Republicans tweet about, and order by the event count

In [11]:
run("""
MATCH (s:Senator {party: "republican"})-[tw:Tweeted]->(t:Tweet)-[m:Mentions]->(e:Entity {type: "EVENT"})
RETURN e.name, count(e)
ORDER BY count(e) DESC
""")

,e.name,count(e)
0,covid,22
1,veterans day,17
2,election day,15
3,halloween,15
4,#,11
...,...,...
273,the wisconsin town association,1
274,oktoberfest,1
275,#veteransday quite like,1
276,the 2022 fall tournament,1


### (9) Get total contributions and total tweet like per senator

In [12]:
run("""
MATCH (c:Contributor)-[d:DonatedTo]->(s:Senator)-[tw:Tweeted]->(t:Tweet)
RETURN s.name as name, sum(d.total) as donations, s.party as party, sum(t.likes) as tweet_likes
ORDER BY sum(t.likes) DESC
""")

,name,donations,party,tweet_likes
0,Mitt Romney,67813200,republican,22513950
1,Bernie Sanders,19227700,other,20708720
2,Marsha Blackburn,107143100,republican,10153120
3,Marco Rubio,71373542,republican,7192900
4,Elizabeth Warren,85030800,democrat,6564280
...,...,...,...,...
71,Jeanne Shaheen,84383046,democrat,40280
72,John Boozman,66459381,republican,38100
73,Mike Crapo,62119332,republican,36220
74,Bill Cassidy,58420250,republican,25380


## --- More Advanced Queries ---

### (10) Get all tweets by Bernie Sanders that mention an organization also mentioned in a rollcall description for which he voted yay on

In [13]:
run("""
MATCH (s:Senator {name: "Bernie Sanders"})-[v:Voted {vote_type: "yea"}]->(r:Rollcall)-[m:Mentions]->(e:Entity {type: "ORG"})
WITH s, e
MATCH (s)-[tw:Tweeted]->(t:Tweet)-[m:Mentions]->(e)
RETURN DISTINCT t.text as tweet, t.likes as likes, e.name as organization
""")

,tweet,likes,organization
0,"Texas, thank you very much for sending @GregCa...",4838,texas
1,"Texas, thank you very much for sending @GregCa...",4838,congress
2,Medicare for All. Tuition-free higher educatio...,1369,texas
3,YOUNG PEOPLE HAVE THE POWER: The younger gener...,1185,texas
4,"In 10 days, we need you, your neighbors, your ...",1347,congress
5,"In 10 days, we need you, your neighbors, your ...",1347,texas
6,Congratulations to @MaxwellFrostFL on his hist...,9195,congress
7,Congratulations to @MaxwellFrostFL on his hist...,9195,washington
8,Congratulations to my new Senate colleague @We...,2722,senate
9,Congratulations to @JasmineForUS on her victor...,4695,texas


### (11) Get the contributers who supported democrat Senators that voted yes on bills mentioning 'the environmental protection agency'. Return the contributor, senator, and the amount.

In [14]:
run("""
MATCH (s:Senator {party: "democrat"})-[v:Voted {vote_type: "yea"}]->(r:Rollcall)-[m:Mentions]->(e:Entity {name: "the environmental protection agency"})
WITH s
MATCH (c:Contributor)-[d:DonatedTo]->(s)
RETURN c.name as contributor, sum(d.total) as amount, s.name as senator
ORDER BY sum(d.total) DESC
""")

,contributor,amount,senator
0,alphabet inc,13015272,Jon Ossoff
1,university of california,8661852,Jon Ossoff
2,emily's list,7250628,Jacky Rosen
3,league of conservation voters,7179830,Jon Tester
4,university of michigan,5435280,Gary Peters
...,...,...,...
755,teaching strategies inc,112000,Jeff Merkley
756,kaiser permanente,111500,Jeff Merkley
757,national education assn,110990,Jeff Merkley
758,american federation of state/cnty/munic employees,109850,Jeff Merkley


### (12) Get the bills that passed within a margin less than 5 and mentioned an EVENT named entity

In [15]:
run("""
MATCH (r:Rollcall)-[m:Mentions]->(e:Entity {type: "EVENT"})
WHERE r.margin < 5
RETURN r.rollnumber as rollnumber, r.desc as description, r.margin as margin, e.name as event_name
ORDER BY r.margin
""")

,rollnumber,description,margin,event_name
0,163,To require a report on State restrictions on r...,1,covid-19
1,806,A bill to improve the Iraq and Afghanistan Ser...,1,iraq and afghanistan service grant
2,807,A bill to improve the Iraq and Afghanistan Ser...,1,iraq and afghanistan service grant
